<a href="https://colab.research.google.com/github/rjzevallos/python-intermedio/blob/main/Clase_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generadores e Iteradores



Recordemos la manera de generar un rango de elementos dentro de Python es mediante la función `range`

In [ ]:
for i in range(1, 4):
    print(i, end=" ")

1 2 3 

Sin embargo, si al crear una instancia de un `range`, el resultado no es una colección con los elementos deseados.

In [ ]:
range(10)

range(0, 10)

El resultado de `range(10)` se conoce como un generador. Pero antes de hablar de generadores es necesario hablar de **iterables**.

## Iterables

Un iterable es un objeto de python que implementa el método `__iter__`. Al utlizar un _for loop_, python llama la función `iter` sobre el objeto a trabajar.

Por abajo del agua, cuando llamamos un for loop,
1. Python llama la función `iter()` sobre el objeto a iterar;
2. La función `iter()` regresa un objeto **iterable** sobre el cuál se define el método `__next__()`;
3. Python llama la el método `__next__` sobre el resultado del `iter` hasta que no existan más elementos a regresar, en cuyo caso Python levanta una excepción `StopIteration` que termina el loop. 

In [ ]:
values = iter(range(1, 4))

In [ ]:
next(values)

1

In [ ]:
next(values)

2

In [ ]:
next(values)

3

<h2 style="color:teal">Ejemplo</h2>

Creemos un primer _iterable_ que nos regrese números entre el 2 y el 10 al azar. Si el número observado es un 1, el ciclo se rompe.

In [ ]:
from random import randint, seed

# Creamos un objeto: necesario para crear un iterable
class RandomValues:
    # Mencionamos que esta clase puede ser iterada
    def __iter__(self):
        return self
    # Definimos que sucede a cada iteración
    def __next__(self):
        value = randint(1, 10)
        if value == 1:
            raise StopIteration  # signals "the end"
        return value

La clase `RandomValues` únicamente implementa los métodos `__iter__` y `__next__`. Necesarios y suficientes para crear un iterable:

In [ ]:
seed(314)
for v in RandomValues():
    print(v, end=" ")

4 8 2 3 

In [ ]:
seed(31415)
for v in RandomValues():
    print(v, end=" ")

10 5 8 

In [ ]:
seed(31415926)
for v in RandomValues():
    print(v, end=" ")

7 7 6 5 8 

<h2 style="color:teal">Ejemplo</h2>
Consideremos el siguiente ejemplo: queremos crear una clase iterable que nos regrese valores de Fibonacci uno a uno:

In [ ]:
class FiboIter:
    # Valores únicos de la clase:
    #  * Número de elementos a iterar
    #  * Número actual (x1)
    #  * Número anterior (x0)
    #  * Número de elementos iterados (curr_elements)
    def __init__(self, n_elements):
        self.x0 = 0
        self.x1 = 1
        self.n_elements = n_elements
        self.curr_elements = 0
    def __iter__(self):
        return self
    def __next__(self):
        self.curr_elements += 1
        if self.curr_elements == 1:
            return self.x0
        elif self.curr_elements == 2:
            return self.x1
        elif self.curr_elements < self.n_elements:
            self.x0, self.x1 = self.x1, self.x0 + self.x1
            return self.x1
        else:
            raise StopIteration

In [ ]:
for n in FiboIter(10):
    print(n, end=" ")

0 1 1 2 3 5 8 13 21 

## Generadores

La complejidad de crear un iterador no es siempre necesaria. Un _generador_ es una simple y efectiva herramienta para crear un iterador.

> Una función _generadora_ nos permite declarar una función que se comporte como un iterador, i.e., que se pueda usar en un _for loop_

La diferencia entre una función y un generador está en usar el keyword `yield` a cada momento que deseamos regresar información.

```python
def generator_fn():
    ...
    yield v
```

A diferencia de una función, al llegar a `yield`, la función regresa un valor y sigue el curso de la función.

In [ ]:
from time import sleep
def give_10():
    for i in range(10):
        yield i

for v in give_10():
    print(v, end=" ")

0 1 2 3 4 5 6 7 8 9 

In [ ]:
from time import sleep
def give_10():
    return [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

for v in give_10():
    print(v, end=" ")

0 1 2 3 4 5 6 7 8 9 

<h2 style="color:teal">Ejemplo</h2>

1. Crea el generador `fizz_buzz(n)` el cual itere `n` veces y cumpla que, para cada `i = 1, ..., n`,
    * Si `i` es divisible por `3`, el generador deberá regresar `"Fizz"`;
    * si `i` es divisible por `5`, el generador deberá regresar `"Buzz"`; 
    * si `i` es divisible por `5` y `3`, el generador deberá regresar `"FizzBuzz"`; 
    * si ninguna de las reglas de arriba se cumple, el generador deberá regresar `i`.

In [ ]:
def fizz_buzz(n):
    for i in range(1, n + 1):
        f_or_b = [i % 3 == 0, i % 5 == 0]
        if any(f_or_b):
            yield "Fizz" * f_or_b[0] + "Buzz" * f_or_b[1]
        else:
            yield i

for v in fizz_buzz(15):
    print(v, end="   ")

1   2   Fizz   4   Buzz   Fizz   7   8   Fizz   Buzz   11   Fizz   13   14   FizzBuzz   

2. Calculemos la suma de los números $\{0, 1, \ldots, 10^{10} - 1\}$. Considerando los siguientes dos programas, **¿Cuál de las siguientes gráficas es más factible haya sido creado por el primer programa? ¿por qué?**

* `Programa_1`
```python
    sum(list(range(10 ** 10)))
```

* `Programa_2`
```python
    sum(range(10 ** 10))
```

Del ejemplo anterior,
* el incremento de memoría usada del segundo al primer programa, en su punto máximo es 9,900% (approx);
* el incremento del primer al segundo programa fue 125% (approx)

Usar un generador es más convieniente cuando:
* Trabajemos con datos sobre los cuáles vamos a iterar. _Desempacar_ los valores uno a uno o _one shot_ depende de la cantidad de información con la que trabajaremos;
* exista un proceso dentro de nuestro programa el cuál no dependa de nuestra computadora, e.g., requerir información de un servidor o base de datos.



## Itertools

`itertools` es una colección de generadores es que regresan `iterators`. Como hemos visto, es importante crear iteradores que controlen el uso de memoria. `itertools` es una librería especializada para tratar con esos problemas.

In [ ]:
import itertools as it
import math

---
### `itertools.count`
Esta función es un contador infinito de valores

In [ ]:
for x in it.count(start=0, step=math.pi / 5):
    y = math.sin(x)
    print(y)
    if round(y) == 0 and x != 0:
        break

0.0
0.5877852522924731
0.9510565162951535
0.9510565162951536
0.5877852522924732
1.2246467991473532e-16


In [ ]:
values = [1, 2, 3, 4]
for v, dec in zip(values, it.count(0, 0.1)):
    print(f"{v} ^ {dec:0.2f} = {v ** dec:0.2f}")

1 ^ 0.00 = 1.00
2 ^ 0.10 = 1.07
3 ^ 0.20 = 1.25
4 ^ 0.30 = 1.52


----
### `itertools.cycle`
Dado un iterable, esta función itera uno a uno los valores y, una vez agotados, incia de nuevo

In [ ]:
letters = ["A", "B"]
for n, letter in enumerate(it.cycle(letters)):
    if n == 10:
        break
    print(n, letter)

0 A
1 B
2 A
3 B
4 A
5 B
6 A
7 B
8 A
9 B


In [ ]:
numbers = range(1, 20)
forms = ["impar", "par"]
for number, form in zip(numbers, it.cycle(forms)):
    print(f"{number:02} {form}")

01 impar
02 par
03 impar
04 par
05 impar
06 par
07 impar
08 par
09 impar
10 par
11 impar
12 par
13 impar
14 par
15 impar
16 par
17 impar
18 par
19 impar


Los iteradores anteriores son conocidos como **iteradores infinitos** dado que no se establece dentro de estos el momento último en el que acaban.

### `itertools.accumulate`
Esta función acumula los resultados de los valores uno a uno hasta que se agote la lista de los valores a iterar. 

La función predeterminada a ocupar es suma. En cuyo caso, dada una lista $\{x_1, x_2, \ldots, x_n\}$ los elementos resultantes serían
$$
    \{x_1, x_1 + x_2, \ldots, x_1 + x_2 + \ldots + x_n\}
$$

In [ ]:
x = [1, 2, 3, 4, 5]
for v in it.accumulate(x):
    print(v, end=" ")

1 3 6 10 15 

la función `accumulate` lleva como parámetro opcional la función a aplicar a cada uno de los elementos. Esta función deberá tomar como primer parámetro el valor acumulado y como segundo parámetro el nuevo valor a aplicar.

In [ ]:
numbers = [-56, -18,  49,  0, -55,  30, -46, -80, -42,  89,  88,  82, -73, -25,  32]
[n for n in it.accumulate(numbers, max)]

[-56, -18, 49, 49, 49, 49, 49, 49, 49, 89, 89, 89, 89, 89, 89]

In [ ]:
[n for n in it.accumulate(numbers, min)]

[-56, -56, -56, -56, -56, -56, -56, -80, -80, -80, -80, -80, -80, -80, -80]

----
### `itertools.chain`
Esta función crea un iterable de elementos iterables sobre una lista

In [ ]:
for v in it.chain(["A", "B"], ["C", "D"]):
    print(v)

A
B
C
D


In [ ]:
for v in it.accumulate(it.chain(["A", "B"], ["C", "D"])):
    print(v)

A
AB
ABC
ABCD


En el caso de tener una lista con iterables sobre los cuáles aplicaremos `chain`, podemos

In [ ]:
# Desempacar los valores de la lista 
letters = [["A", "B"], ["C", "D"]]
for v in it.chain(*letters):
    print(v)

A
B
C
D


In [ ]:
# usar la propiedad "from_iterable" dentro de letters
letters = [["A", "B"], ["C", "D"]]
for v in it.chain.from_iterable(letters):
    print(v)

A
B
C
D


----
### `itertools.groupby`
Esta función crea un iterador que regresa las llaves de agrupación y los elementos agrupados dentro de un conjunto de datos.

In [ ]:
values = "AAAAAXXBBBBCCCCCDDDOOOO"
for key, iterables in it.groupby(values):
    print(key)

A
X
B
C
D
O


In [ ]:
values = "AAAAABBBBCCCCCDDDOOOO"
for key, iterables in it.groupby(values):
    print(iterables)

----
### `itertools.compress`
Crea un iterador que regresa únicamente los elementos que regresen ciertos.

In [ ]:
values = [3, 1, 4, 1, 5, 2, 3, 1, 5, 2]
select = [1, 0, 0, 0, 1, 0, 1, 0, 0, 1]

for v in it.compress(values, select):
    print(v)

3
5
3
2


<h2 style="color:teal">Ejemplo</h2>

Crea la función `range_primes` que regrese todos los números primos entre un rango `[2, n]`.

In [ ]:
def range_primes(values, primes=None):
    primes = primes if primes is not None else [2]
    *_, last_prime = primes
    prime, *values = it.compress(values, map(lambda x: x % last_prime != 0, values))
    primes.append(prime)
    if len(values) != 0:
        return range_primes(values, primes)
    else:
        return primes

In [ ]:
vals = range(2, 60)
range_primes(vals)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59]

----
### `itertools.product`
Esta función regresa el producto cartesiano entre dos iterables. Esta  función es comunmente usada como reemplazo de un doble for loop.

In [ ]:
upper = ["A", "B", "C"]
lower = ["a", "b", "c"]

for u, l in it.product(upper, lower):
    print(f"{u}-{l}", end="  ")

A-a  A-b  A-c  B-a  B-b  B-c  C-a  C-b  C-c  

In [ ]:
table_size = 5
elements = [*[" "] * (table_size - 1), "\n"]
for (i, j), end in zip(it.product(*it.repeat(range(1, table_size + 1), 2)), it.cycle(elements)):
    print(f"{i * j:02}", end=end)

01 02 03 04 05
02 04 06 08 10
03 06 09 12 15
04 08 12 16 20
05 10 15 20 25


----
### `itertools.permutations`
Esta función crea un iterable de todas las permutaciones posibles de elementos de longitud `m`

In [ ]:
for v in it.islice(it.permutations("Nabla"), 10):
    print(v)

('N', 'a', 'b', 'l', 'a')
('N', 'a', 'b', 'a', 'l')
('N', 'a', 'l', 'b', 'a')
('N', 'a', 'l', 'a', 'b')
('N', 'a', 'a', 'b', 'l')
('N', 'a', 'a', 'l', 'b')
('N', 'b', 'a', 'l', 'a')
('N', 'b', 'a', 'a', 'l')
('N', 'b', 'l', 'a', 'a')
('N', 'b', 'l', 'a', 'a')


----
### `itertools.combinations`
Esta función crea un iterable de todas las combinaciones posibles de elementos de longitud `m`

In [ ]:
for v in it.combinations("Nabla", 4):
    print(v)

('N', 'a', 'b', 'l')
('N', 'a', 'b', 'a')
('N', 'a', 'l', 'a')
('N', 'b', 'l', 'a')
('a', 'b', 'l', 'a')


## Referencias
1. https://wiki.python.org/moin/Generators
2. https://wiki.python.org/moin/Iterator
3. https://docs.python.org/3/tutorial/classes.html
4. https://docs.python.org/3/library/itertools.html